# Response of the Smallbone model to step function input

Purpose of this experiment: to verify that the Smallbone model in python has the same response as the MATLAB model.

<img src="metabolites-time_matlab.png" alt="metabolites vs time in matlab" width="50%"/><img src="fluxes-time_matlab.png" alt="fluxes vs time in matlab" width="50%"/>

## The structure of the Trehalose cycle model

In [1]:
import libsbml
sbml = libsbml.readSBML("../../data/trehalose/smallbone.xml")
smallbone = sbml.model

In [2]:
from pprint import pprint

**Unit definition**

In [3]:
unit_definitions = list(smallbone.unit_definitions)
pprint(unit_definitions)

[<UnitDefinition substance "mmol">,
 <UnitDefinition time "min">,
 <UnitDefinition mM "mM">,
 <UnitDefinition per_min "per min">,
 <UnitDefinition mM_per_min "mM per min">]


In [4]:
mmol = unit_definitions[0]

In [5]:
m = list(mmol.getListOfUnits())[0]
print(libsbml.UnitKind_toString(m.kind))
print(libsbml.UnitDefinition_printUnits(unit_definitions[1]))

mole
second (exponent = 1, multiplier = 60, scale = 0)


**Compartment**

In [6]:
compartments = list(smallbone.compartments)
pprint(compartments)

[<Compartment cell "cell">, <Compartment medium "medium">]


**Reactions**

In [23]:
reactions = list(smallbone.reactions)
pprint(reactions)

[<Reaction pgi "G6P isomerase">,
 <Reaction hxt "glucose transport">,
 <Reaction hxk "hexokinase">,
 <Reaction pgm "phosphoglucomutase">,
 <Reaction tpp "T6P phosphatase">,
 <Reaction tps "T6P synthase">,
 <Reaction nth "trehalase">,
 <Reaction ugp "UDP glucose phosphorylase">]


In [24]:
r = reactions[0]
kl = r.getKineticLaw()
print(kl.formula)
libsbml.UnitDefinition.printUnits(kl.getDerivedUnitDefinition())

cell * pow(shock, heat) * Vmax / Kg6p * (g6p - f6p / Keq) / (1 + g6p / Kg6p + f6p / Kf6p)


'second (exponent = -1, multiplier = 60, scale = 0), mole (exponent = 1, multiplier = 0.001, scale = 0)'

This is correct because flux should be a $mole/sec$ unit

Every node in the sbml document also has the *reference* to the entire model👇

In [25]:
r.model

<Model Smallbone2011_TrehaloseBiosynthesis "Smallbone2011_TrehaloseBiosynthesis">

In [26]:
lr=reactions[0].getListOfReactants()

In [28]:
sr = lr[0]
print(type(sr))
print(sr.stoichiometry)

<class 'libsbml.SpeciesReference'>
1.0


**Species**

In [29]:
species = list(smallbone.species)
pprint(species)

[<Species glc "glucose">,
 <Species g1p "glucose 1-phosphate">,
 <Species g6p "glucose 6-phosphate">,
 <Species trh "trehalose">,
 <Species t6p "trehalose 6-phosphate">,
 <Species udg "UDP glucose">,
 <Species adp "ADP">,
 <Species atp "ATP">,
 <Species ppi "diphosphate">,
 <Species f6p "fructose 6-phosphate">,
 <Species h "H+">,
 <Species pho "phosphate">,
 <Species udp "UDP">,
 <Species utp "UTP">,
 <Species h2o "water">,
 <Species glx "glucose">]


In [30]:
s = species[0]
i = 0
print("Species " + str(i) + ": "
    + libsbml.UnitDefinition.printUnits(s.getDerivedUnitDefinition()))

Species 0: mole (exponent = 1, multiplier = 1, scale = -3), litre (exponent = -1, multiplier = 1, scale = 0)


**Parameters**

In [42]:
parameters = list(smallbone.parameters)
pprint(parameters)

[<Parameter heat "heat">,
 <Parameter glc_0>,
 <Parameter glc_change "log10 change in glucose">,
 <Parameter g1p_0>,
 <Parameter g1p_change "log10 change in glucose 1-phosphate">,
 <Parameter g6p_0>,
 <Parameter g6p_change "log10 change in glucose 6-phosphate">,
 <Parameter trh_0>,
 <Parameter trh_change "log10 change in trehalose">,
 <Parameter t6p_0>,
 <Parameter t6p_change "log10 change in trehalose 6-phosphate">,
 <Parameter udg_0>,
 <Parameter udg_change "log10 change in UDP glucose">]


**Parameter 2**
```xml
<parameter
          metaid="_345287"
          id="glc_change"
          name="log10 change in glucose"
          units="dimensionless"
          constant="false" />
```

In [43]:
print(parameters[2].value,
    parameters[2].id,
    parameters[2].constant,
    parameters[2].units
)

0.0 glc_change False dimensionless


In [44]:
rules = smallbone.rules
for r in rules:
    print(r.variable, "=", libsbml.formulaToString(r.math))

glc_change = log10(glc / glc_0)
g1p_change = log10(g1p / g1p_0)
g6p_change = log10(g6p / g6p_0)
trh_change = log10(trh / trh_0)
t6p_change = log10(t6p / t6p_0)
udg_change = log10(udg / udg_0)


In [45]:
rules.get('glc_0') is None

True

**Initial assignments**

Assign any symbol in the model with the formula when initializing the model. 

This will overwrite the `initial_concentration` property of species if the `symbol` is a species

For example, the following `initialAssignment` initializes glucose `glc` with parameter `glc_0`

```xml
<initialAssignment metaid="_949171" symbol="glc">
    <math xmlns="http://www.w3.org/1998/Math/MathML">
        <ci> glc_0 </ci>
    </math>
</initialAssignment>
```

Even if species `glc` has `initial_concentration` property "0.09675"

```xml
<species
          metaid="metaid_glc"
          id="glc"
          name="glucose"
          compartment="cell"
          initialConcentration="0.09675"
          sboTerm="SBO:0000247">
```

In [48]:
ias = smallbone.initial_assignments
for iar in ias:
    print(iar.symbol, "<-", libsbml.formulaToString(iar.math))

glc <- glc_0
g1p <- g1p_0
g6p <- g6p_0
trh <- trh_0
t6p <- t6p_0
udg <- udg_0


In [50]:
ias.get('H') is None

True

In [51]:
from pyADAPT.bio import SBMLModel

%pylab inline
# %config InlineBackend.figure_format = 'retina'

Populating the interactive namespace from numpy and matplotlib


First, set the initial condition of the system to a steady states.

In [ ]:
x = list()
for s in smallbone.model.getListOfSpecies():
    x.append(s.initial_concentration)
x = array(x)

t_eval = np.linspace(0, 10, 50)
y = smallbone.compute_states([0, 10], x, t_eval=t_eval)

for i in range(y.shape[0]):
    plot(t_eval, y[i, :], "--")
names = [x.name for x in smallbone.sbml_species_list]
legend(names);